# The Cat Expert Bot

Install the nesesary stuff

In [ ]:
#%pip install pytube
#%pip install --upgrade pytube
#%pip install yt-dlp
#%pip install moviepy
#%pip install whisper

#%pip install git+https://github.com/openai/whisper.git
#%pip install pytubefix
#%pip install chromadb
#%pip install langchain
#%pip install openai


Load the videos to learn.

In [2]:
from pytubefix import YouTube

# Replace with your list of YouTube URLs
video_urls = ["https://youtu.be/ZUcVUFvmDFE?si=z9GfOAWF1qothiKs", 
            "https://youtu.be/4DlJYcfiRu4?si=cUVT9L5dEEdkcSt_", 
            "https://youtu.be/rxInrRQLEmM?si=Ai7wHN0dI--cns0x", 
            "https://youtu.be/gxlNfh5ukMw?si=naO3n4VZeXx3PlOs", 
            "https://youtu.be/ojS7XwtoXtw?si=NpNSef7dCm_LnFPv", 
            "https://youtu.be/tsYT7yIOdqQ?si=hdGEpxlmFNMf7NNQ", 
            "https://youtu.be/tsYT7yIOdqQ?si=e_Zdh2dGpqempHR8", 
            "https://youtu.be/UWohxDOXsl4?si=y1nXlUZYw6uzkc8n", 
            "https://youtu.be/gZrwcoiy_gY?si=ksfYE03t6xtuxUL0", 
            "https://youtu.be/lSDI5diNu4Y?si=Q-In6zMD4ZpuaPIz", 
            "https://youtu.be/8aCGL9GpVUg?si=_0yF1U1thjwJqyPY", 
            "https://youtu.be/VjOXvD7OvrE?si=t6xugNxLeMjpsi7E", 
            "https://youtu.be/FzifwTnCV5s?si=sR_u4kG-4NoQx5Ux", 
            "https://youtu.be/XreeFU7RYeI?si=hsc9WO24dJP6AfV2", 
            "https://youtu.be/-4O97jw_8Bc?si=pC14dgZ_f4mXdYPv" ]

for url in video_urls:
    yt = YouTube(url)
    video = yt.streams.filter(only_audio=True).first()
    video.download(output_path="path/to/m4a")


import yt_dlp

# Define the playlist URL and the output path for saving audio files
playlist_url = "https://www.youtube.com/playlist?list=PLAJvHNBwbBNvSQFSeeHswdgIk2m4fljOc"
output_path = "path/to/save/%(title)s.%(ext)s"  # Customize as needed

# Set options for downloading only the audio
ydl_opts = {
    'format': 'bestaudio/best',       # Best available audio quality
    'outtmpl': output_path,           # Output file naming format
    'postprocessors': [{              # Convert to mp3 if desired
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
    'ignoreerrors': True,             # Continue to next video if an error occurs
}

# Download audio for each video in the playlist
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([playlist_url])


In [3]:
#import whisper
#import glob

#model = whisper.load_model("base")

# Dynamically list all .m4a files in a directory
#audio_files = glob.glob("/notebooks/path/to/m4a/*.m4a")

#for audio_file in audio_files:
#    # Transcribe directly from .m4a
#    transcription = model.transcribe(audio_file)
#    with open(audio_file.replace(".m4a", ".txt"), "w") as f:
#        f.write(transcription['text'])

# Transcriptions done by Whisper

In [4]:
import whisper
import os

# Load Whisper model
model = whisper.load_model("base")

# Directory containing your .m4a files
audio_dir = "/notebooks/path/to/m4a/"
transcription_dir = "/notebooks/path/to/transcriptions/"

# Ensure the transcription directory exists
os.makedirs(transcription_dir, exist_ok=True)

# List all .m4a files in the audio directory
audio_files = [f for f in os.listdir(audio_dir) if f.endswith(".m4a")]

# Transcribe each audio file and save it as a .txt file
for audio_file in audio_files:
    audio_path = os.path.join(audio_dir, audio_file)
    transcription = model.transcribe(audio_path)
    
    # Save transcription to a .txt file
    transcription_file = os.path.join(transcription_dir, audio_file.replace(".m4a", ".txt"))
    with open(transcription_file, "w") as f:
        f.write(transcription['text'])

    print(f"Transcribed and saved: {transcription_file}")


Transcribed and saved: /notebooks/path/to/transcriptions/When Cat Introductions Get UGLY.txt
Transcribed and saved: /notebooks/path/to/transcriptions/Can my Cats Get Along? Cat-to-Cat Body Language basics & Introduction Tips.txt
Transcribed and saved: /notebooks/path/to/transcriptions/How To Introduce Your Cat to a New KITTEN!.txt
Transcribed and saved: /notebooks/path/to/transcriptions/How to Introduce Cats.txt
Transcribed and saved: /notebooks/path/to/transcriptions/Are My Cats Playing or Fighting? | Cat Playing vs Cat Aggression.txt
Transcribed and saved: /notebooks/path/to/transcriptions/Cat Introductions Gone Wrong: They Will NOT Work it Out Without You.txt
Transcribed and saved: /notebooks/path/to/transcriptions/This was BAD!! 🙀🤯🫣 #shorts.txt
Transcribed and saved: /notebooks/path/to/transcriptions/The Do's & Don'ts of Introducing Cats.txt
Transcribed and saved: /notebooks/path/to/transcriptions/Cat Introductions: Good First Impressions are a must!.txt
Transcribed and saved: /not

# Chromadb as Client and creation of Collection

In [5]:
import chromadb
from sentence_transformers import SentenceTransformer
import glob

# Initialize ChromaDB client and create a collection
client = chromadb.Client()
if "cat_expert_knowledge" in [col.name for col in client.list_collections()]:
    client.delete_collection("cat_expert_knowledge")
collection = client.create_collection("cat_expert_knowledge")

# Load the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Load and process each transcription file
for idx, transcription_file in enumerate(glob.glob("/notebooks/path/to/transcriptions/*.txt")):
    with open(transcription_file, "r") as f:
        text = f.read()

    # Create an embedding for the transcription
    embedding = model.encode(text).tolist()
    topic = os.path.basename(transcription_file).replace(".txt", "")  # Use file name as topic

    # Add to ChromaDB collection with a unique ID
    collection.add(
        ids=[f"doc_{idx}"],  # Unique ID for each document
        documents=[text],
        metadatas=[{"topic": topic, "source": transcription_file}],
        embeddings=[embedding]
    )

print("Transcriptions successfully stored in ChromaDB!")

Transcriptions successfully stored in ChromaDB!


# Querying Chromadb

In [6]:
# Sample query to retrieve relevant information
query_text = "How do I stop my cat from scratching furniture?"
query_embedding = model.encode(query_text).tolist()

# Perform the search
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3  # Number of top results to retrieve
)

# Display results
for result in results['documents']:
    print("Retrieved Text:", result)
for metadata in results['metadatas']:
    print("Metadata:", metadata)


Retrieved Text: [" What's up gang is I Jackson Galaxy your cat daddy and today your cat question answerer that is what I'm here for today hopefully we get questions all the time from folks just like you who sent it into this address right there which you of course can do as well and today we're gonna talk to Kinesha Kinesha what's it all about hi Jackson I'm Kinesha from the Cayman Islands my husband and I have a cat sunshine she's 11 years old and we've always wanted to get another cat to you know keep her company and also rescue from the human society but you know we're a bit worried because she has been the only cat in the home for so long in 11 years and we just wonder whether or not she'll be very receptive to the addition of another cat in the home we'd love your feedback on the best approach to do so you know what can we do to make this a seamless process because we'd really love for it to be a success and to avoid any elimination or scratching or you know any of the behaviors t

In [7]:
#%pip install langchain
#%pip install openai

In [ ]:
from transformers import GPT2Tokenizer
from langchain.chat_models import ChatOpenAI  # Ensure ChatOpenAI is imported correctly
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from sentence_transformers import SentenceTransformer
import chromadb
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv("apikey.env")

# Access the API key from environment variables
api_key = os.getenv("OPENAI_API_KEY")
if api_key is None:
    raise ValueError("OPENAI_API_KEY is missing. Please set it in your .env file.")

# Initialize ChatOpenAI model with the API key for chat-based models
llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=api_key)

# Initialize ChromaDB client and retrieve the collection
client = chromadb.Client()
collection = client.get_collection("cat_expert_knowledge")

# Initialize the embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize a tokenizer to estimate token counts
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Limit context length by token count (e.g., 1500 tokens for context)
MAX_CONTEXT_TOKENS = 1500

# Multi-query generation: create multiple query variations
def generate_query_variations(query):
    """Create variations of a query to enhance retrieval diversity."""
    return [
        query,
        f"Explain about {query}",
        f"How does {query} work?",
        f"What are the details of {query}?",
        f"Tell me more about {query}"
    ]

# Updated retrieve_documents function with token count management
def retrieve_documents(query):
    query_variations = generate_query_variations(query)
    results = []

    for variation in query_variations:
        query_embedding = embedding_model.encode(variation).tolist()
        result = collection.query(query_embeddings=[query_embedding], n_results=2)
        documents = [doc if isinstance(doc, str) else " ".join(doc) for doc in result['documents']]
        results.extend(documents)

    # Combine all retrieved documents
    full_context = " ".join(results)

    # Tokenize and truncate if the context exceeds the maximum token limit
    tokens = tokenizer.encode(full_context)
    if len(tokens) > MAX_CONTEXT_TOKENS:
        tokens = tokens[:MAX_CONTEXT_TOKENS]
        truncated_context = tokenizer.decode(tokens)
    else:
        truncated_context = full_context

    return truncated_context

# Define the LangChain ChatPromptTemplate with messages for chat models
def create_qa_chain():
    # Chat prompt template for structured input to chat model
    template = """You are a friendly, expert cat behaviorist inspired by Jackson Galaxy.
Use the information below to answer the question comprehensively."""

    prompt = ChatPromptTemplate.from_messages([
        ("system", template),
        ("user", "{context}\n\nQuestion: {question}\nAnswer:")
    ])

    # Define the LLMChain with the chat prompt for generating answers
    return LLMChain(llm=llm, prompt=prompt)

# Main agent to respond to user queries
context_history = []

def ask_agent(query):
    # Append the new question to context history
    context_history.append(f"Question: {query}")
    
    # Retrieve relevant context from ChromaDB based on query variations
    conversation_context = retrieve_documents(query)
    
    # Create QA chain with the generated context
    qa_chain = create_qa_chain()
    answer = qa_chain.run({"context": conversation_context, "question": query})

    # Append the answer to the context history for continuity
    context_history.append(f"Answer: {answer}")
    return answer

# Test the agent with a sample query
user_query = "Why does my cat scratch furniture?"
response = ask_agent(user_query)
print("Agent Response:", response)


Token indices sequence length is longer than the specified maximum sequence length for this model (24579 > 1024). Running this sequence through the model will result in indexing errors
2024-10-31 16:47:23.110821: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-31 16:47:23.110940: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-31 16:47:23.210403: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-31 16:47:23.402070: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical opera

Agent Response: Cats scratch furniture for a variety of reasons, and understanding these reasons can help you address this behavior in a positive way. Here are some common reasons why cats scratch furniture:

1. **Territorial Marking**: Cats have scent glands in their paws, and when they scratch, they leave both a visual mark (the scratches) and a scent mark to establish their territory. This behavior is a natural instinct for cats.

2. **Nail Maintenance**: Scratching helps cats remove the outer sheath of their claws, keeping them healthy and sharp. It's a way for them to stretch and exercise their muscles as well.

3. **Stress Relief**: Scratching can also be a way for cats to relieve stress and anxiety. It helps them feel more secure in their environment.

4. **Environmental Enrichment**: Cats scratch to mark their presence, communicate with other cats, and leave their scent in their surroundings. It's a way for them to feel more at home.

To prevent your cat from scratching furnitu

# Isabela fix


In [ ]:
#%pip install langchain_community langchain_openai

In [ ]:
import pydantic 
pydantic.__version__

'2.9.2'

In [ ]:
import langchain 
langchain.__version__

'0.3.6'

In [45]:
# Modify the prompt template to reflect a friendly, expert tone
template = """You are an expert in cat behavior with a friendly and approachable style, similar to Jackson Galaxy, the famous 'Cat Whisperer'. 
Use the information below to provide advice on the question as if you were explaining it to a new cat owner.

{context}

Question: {question}
Answer:"""

prompt = PromptTemplate(input_variables=["context", "question"], template=template)


In [46]:
# Add a context history list to keep track of the conversation context
context_history = []

def ask_agent(query):
    # Append the new question to context history
    context_history.append(f"Question: {query}")

    # Generate query variations
    query_variations = generate_query_variations(query)
    retrieved_docs = retrieve_documents(query_variations)
    context = " ".join(retrieved_docs)  # Combine retrieved text into context

    # Incorporate previous conversation into context
    conversation_context = " ".join(context_history[-3:])  # Keep last 3 exchanges

    # Create updated context for the agent to respond with historical continuity
    qa_chain = create_qa_chain()
    answer = qa_chain.run({"context": conversation_context + "\n" + context, "question": query})

    # Store the answer in context history to maintain continuity
    context_history.append(f"Answer: {answer}")

    return answer


In [ ]:
#%pip install yt-dlp

In [17]:
import yt_dlp
import os

# Define the playlist URL and output path for saving video files
playlist_url = "https://www.youtube.com/playlist?list=PLAJvHNBwbBNvSQFSeeHswdgIk2m4fljOc"  # Replace with actual playlist URL
output_path = "path/to/save/videos/%(title)s.%(ext)s"  # Directory to save videos

# Set yt-dlp options for downloading video
ydl_opts = {
    'format': 'bestvideo+bestaudio',  # Downloads the best quality video and audio
    'outtmpl': output_path,           # Sets the output path and filename format
    'ignoreerrors': True,             # Skip any videos that cause errors
}

# Download videos from the specified playlist
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([playlist_url])

print("Videos have been successfully downloaded.")


[youtube:tab] Extracting URL: https://www.youtube.com/playlist?list=PLAJvHNBwbBNvSQFSeeHswdgIk2m4fljOc
[youtube:tab] PLAJvHNBwbBNvSQFSeeHswdgIk2m4fljOc: Downloading webpage
[youtube:tab] PLAJvHNBwbBNvSQFSeeHswdgIk2m4fljOc: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: INTRODUCTIONS
[youtube:tab] PLAJvHNBwbBNvSQFSeeHswdgIk2m4fljOc page 1: Downloading API JSON


[youtube:tab] PLAJvHNBwbBNvSQFSeeHswdgIk2m4fljOc page 1: Downloading API JSON


[youtube:tab] PLAJvHNBwbBNvSQFSeeHswdgIk2m4fljOc page 1: Downloading API JSON


[youtube:tab] PLAJvHNBwbBNvSQFSeeHswdgIk2m4fljOc page 1: Downloading API JSON


[youtube:tab] Playlist INTRODUCTIONS: Downloading 15 items of 15
[download] Downloading item 1 of 15
[youtube] Extracting URL: https://www.youtube.com/watch?v=ZUcVUFvmDFE
[youtube] ZUcVUFvmDFE: Downloading webpage
[youtube] ZUcVUFvmDFE: Downloading ios player API JSON
[youtube] ZUcVUFvmDFE: Downloading mweb player API JSON
[youtube] ZUcVUFvmDFE: Downloading player 78fcc867
[youtube] ZUcVUFvmDFE: Downloading m3u8 information
[info] ZUcVUFvmDFE: Downloading 1 format(s): 616+251
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 191
[download] Destination: path/to/save/videos/When Cat Introductions Get UGLY.f616.mp4
[download] 100% of  294.23MiB in 00:00:11 at 25.42MiB/s                   
[download] Destination: path/to/save/videos/When Cat Introductions Get UGLY.f251.webm
[download] 100% of   14.30MiB in 00:00:00 at 37.33MiB/s    
[Merger] Merging formats into "path/to/save/videos/When Cat Introductions Get UGLY.webm"
Deleting original file path/to/save/videos/When Cat I

In [14]:
%pip install opencv-python

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import os

# Directory containing videos
video_dir = "/path/to/save/videos/"
frame_dir = "/path/to/frames/"

os.makedirs(frame_dir, exist_ok=True)

def extract_key_frames(video_path, output_folder, intervals=60):
    """Extract frames at given intervals (in seconds) from a video."""
    video_name = os.path.basename(video_path).split('.')[0]
    capture = cv2.VideoCapture(video_path)
    fps = capture.get(cv2.CAP_PROP_FPS)
    interval_frames = int(fps * intervals)

    success, frame = capture.read()
    count = 0
    while success:
        if count % interval_frames == 0:
            frame_name = f"{video_name}_frame_{count}.jpg"
            cv2.imwrite(os.path.join(output_folder, frame_name), frame)
        success, frame = capture.read()
        count += 1
    capture.release()

# Extract frames from each video
for video_file in os.listdir(video_dir):
    extract_key_frames(os.path.join(video_dir, video_file), frame_dir)


FileNotFoundError: [Errno 2] No such file or directory: '/path/to/videos/'